In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_read
from audiocraft.data.audio_utils import convert_audio
import torch.nn.functional as F

objc[94863]: Class AVFFrameReceiver is implemented in both /Users/epple/audio_craft/.env/lib/python3.9/site-packages/av/.dylibs/libavdevice.60.3.100.dylib (0x1570b0760) and /opt/homebrew/Cellar/ffmpeg/6.1.1_6/lib/libavdevice.60.3.100.dylib (0x16b99c370). One of the two will be used. Which one is undefined.
objc[94863]: Class AVFAudioReceiver is implemented in both /Users/epple/audio_craft/.env/lib/python3.9/site-packages/av/.dylibs/libavdevice.60.3.100.dylib (0x1570b07b0) and /opt/homebrew/Cellar/ffmpeg/6.1.1_6/lib/libavdevice.60.3.100.dylib (0x16b99c3c0). One of the two will be used. Which one is undefined.
/Users/epple/audio_craft/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Encode with Encodec_32khz from Facebook
audio_path = 'dataset/example/electro_1.mp3'

out, sr = audio_read(audio_path)

In [10]:
out = convert_audio(out, from_rate = sr, to_rate = 32000, to_channels=1)
segment_duration = 30
sample_rate = 32000
target_frames = int(segment_duration * sample_rate)
n_frames = out.shape[-1]
out = F.pad(out, (0, target_frames - n_frames))

In [13]:
out.unsqueeze(0).shape

torch.Size([1, 1, 960000])

In [2]:
musicgen = MusicGen.get_pretrained('checkpoints/my_audio_lm')

/Users/epple/audio_craft/.env/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [30]:
out = out.reshape(1,-1)
audio_tokens, scale = musicgen.compression_model.encode(out.unsqueeze(0))

EncodecModel(
  (encoder): EncodecEncoder(
    (layers): ModuleList(
      (0): EncodecConv1d(
        (conv): Conv1d(1, 64, kernel_size=(7,), stride=(1,))
      )
      (1): EncodecResnetBlock(
        (block): ModuleList(
          (0): ELU(alpha=1.0)
          (1): EncodecConv1d(
            (conv): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
          )
          (2): ELU(alpha=1.0)
          (3): EncodecConv1d(
            (conv): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
          )
        )
        (shortcut): Identity()
      )
      (2): ELU(alpha=1.0)
      (3): EncodecConv1d(
        (conv): Conv1d(64, 128, kernel_size=(8,), stride=(4,))
      )
      (4): EncodecResnetBlock(
        (block): ModuleList(
          (0): ELU(alpha=1.0)
          (1): EncodecConv1d(
            (conv): Conv1d(128, 64, kernel_size=(3,), stride=(1,))
          )
          (2): ELU(alpha=1.0)
          (3): EncodecConv1d(
            (conv): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  

In [16]:
audio_tokens

tensor([[[ 648,  118, 2008,  ...,   83,   83,   83],
         [1005, 1070, 1048,  ..., 2044, 2044, 2044],
         [ 551,  638, 1854,  ..., 2019, 2019, 2019],
         [ 281, 1795, 1661,  ..., 1770, 1770, 1770]]])